In [184]:
import pandas as pd

In [185]:
b_product = pd.read_csv(r'data\branch_products.csv', encoding='windows-1251')
products = pd.read_csv('data\products.csv', encoding='windows-1251')
rc_products = pd.read_csv(r'data\rc_products.csv', encoding='windows-1251')

In [ ]:
# b_product = b_product[:3000]
# rc_products = rc_products[:3000]

In [186]:
# в процессе работы заметила, что некоторые товары в таблицах продуктов отсутсвуют, поэтому добавила проверку 
# на совпадение данных, а в ячейке ниже удалила отсутствующие строки
missing_in_bproducts = set(b_product['Товар']) - set(products['Product_ID'])
print(f"товары из branch_products, отсутствующие в продуктах: {len(missing_in_bproducts)}")

missing_in_rc = set(rc_products['Товар']) - set(products['Product_ID'])
print(f"товары из rc_products, отсутствующие в продуктах: {len(missing_in_rc)}")

товары из branch_products, отсутствующие в продуктах: 4345
товары из rc_products, отсутствующие в продуктах: 2477


In [187]:
valid_products = set(products['Product_ID'])
b_product = b_product[b_product['Товар'].isin(valid_products)]
rc_products = rc_products[rc_products['Товар'].isin(valid_products)]

In [203]:
import psycopg2
from psycopg2 import sql

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="11111",
    host="127.0.0.1",
    port="5432"
)
cursor = conn.cursor()

In [ ]:
def create_tables():

    cursor.execute('''create table products (
    product_id uuid primary key,
    category_id uuid not null
    );''')

    cursor.execute('''create table stores (
    branch_id uuid primary key,
    priority numeric not null
    );''')

    cursor.execute('''create table logdays (
    branch_id uuid references stores(branch_id),
    category_id uuid,
    logdays numeric default 7,
    primary key (branch_id, category_id)
    );''')

    cursor.execute('''create table needs (
    branch_id uuid references stores(branch_id),
    product_id uuid references products(product_id),
    needs numeric not null,
    primary key (branch_id, product_id)
    );''')

    cursor.execute('''create table branch_products (
    product_id uuid references products(product_id),
    branch_id uuid references stores(branch_id),
    residue numeric not null check (residue >= 0),
    reserve numeric not null check (reserve >= 0),
    transit numeric not null check (transit >= 0),
    primary key (product_id, branch_id)
    );''')

    # заметила аномалию в таблице rc_products(отрицательное значение), поэтому ограничение на отрицательное значение сняла, 
    # для корректного вывода алгоритма
    cursor.execute('''create table rc_products (
    product_id uuid references products(product_id),
    rc uuid not null, 
    residue numeric not null,
    reserve numeric not null check (reserve >= 0),
    transit numeric not null check (transit >= 0),
    primary key (product_id, rc)
    );''')

    conn.commit()
    print('таблицы созданы')

create_tables()

таблицы созданы


In [ ]:
import random

def insert_tables():

    with conn.cursor() as cursor:
        print(1)
        for _, row in products.iterrows():
            cursor.execute('''insert into products (product_id, category_id) values(%s, %s)''',
                        (row['Product_ID'], row['Category_ID']))
        conn.commit()

    unique_stores = b_product['Фирма'].unique()
    
    with conn.cursor() as cursor:
        print(2)
        for store_id in unique_stores:
            priority = random.randint(1, 5) 
            cursor.execute(
                "insert into stores (branch_id, priority) values (%s, %s)",
                (store_id, priority)
            )
        conn.commit()
    
    unique_category = products['Category_ID'].unique()

    with conn.cursor() as cursor:
        print(3)
        for store_id in unique_stores:
            for category_id in unique_category:
                cursor.execute(
                    "insert into logdays (branch_id, category_id) values (%s, %s) on conflict do nothing",
                    (store_id, category_id)
                )
        conn.commit()

    with conn.cursor() as cursor:
        print(4)
        for _, row in b_product.iterrows():
            cursor.execute('''insert into branch_products (product_id, branch_id, residue, reserve, transit) 
                values (%s, %s, %s, %s, %s) on conflict do nothing''',
                        (row['Товар'], row['Фирма'], row['Остаток'], row['Резерв'], row['Транзит']))
        conn.commit()

    with conn.cursor() as cursor:
        print(5)
        for _, row in rc_products.iterrows():
            cursor.execute('''insert into rc_products (product_id, rc, residue, reserve, transit) 
                values (%s, %s, %s, %s, %s) on conflict do nothing''',
                        (row['Товар'], row['РЦ'], row['Остаток'], row['Резерв'], row['Транзит']))
        conn.commit()

In [199]:
insert_tables()

5


In [200]:
# дополнительно(параметр min_shipment)
query = """
alter table needs add column min_shipment numeric;
"""
cursor.execute(query)
conn.commit() 

DuplicateColumn: column "min_shipment" of relation "needs" already exists


In [204]:
def insert_needs():
    with conn.cursor() as cursor:
        cursor.execute("""
        SELECT bp.product_id, bp.branch_id, bp.residue, p.category_id
        FROM branch_products bp
        JOIN products p ON bp.product_id = p.product_id
        """)
        branch_products = cursor.fetchall()
        
        for product_id, branch_id, residue, category_id in branch_products:
            cursor.execute("""
            SELECT logdays FROM logdays 
            WHERE branch_id = %s AND category_id = %s
            """, (branch_id, category_id))
            logdays = cursor.fetchone()[0]
            
            needs = random.randint(1, max(150, residue)) * logdays
            # расчет параметра min_shipment считается исходя из спроса магазина и срока доставки,
            # так так в данном случае доставка всегда 7 дней, то needs = min_shipment
            min_shipment = needs*(logdays/7)

            cursor.execute("""
            INSERT INTO needs (branch_id, product_id, needs, min_shipment) 
            VALUES (%s, %s, %s, %s)
            """, (branch_id, product_id, needs, min_shipment))
        
        conn.commit()

In [ ]:
insert_needs()

In [ ]:
def distribution_algorithm():

    query = '''
    select p.product_id, p.category_id,s.branch_id,s.priority,
    bp.residue as current_count,
    n.needs,n.min_shipment, bp.transit, l.logdays,
    coalesce (rc.residue - rc.reserve, 0) as available
    from products p
    cross join stores s 
    left join branch_products bp on bp.product_id = p.product_id and bp.branch_id = s.branch_id
    left join needs n on n.product_id = p.product_id and n.branch_id = s.branch_id
    left join logdays l on l.branch_id = s.branch_id and l.category_id = p.category_id
    left join rc_products rc on rc.product_id = p.product_id
    where n.needs > 0 or (bp.product_id is not null and bp.branch_id is not null)
    '''

    data = pd.read_sql(query, conn)

    # расчет дефицита товара
    def calculate_deficit(row):
        needed_amount = row['min_shipment']
        total_available = row['current_count'] + row['transit']
        deficit = max(needed_amount - total_available, 0) 
        return deficit
    
    data['deficit'] = data.apply(lambda row: calculate_deficit(row), axis=1)
    # фильтруем строки и оставляем только те, где положительные значения, затем сортируем по большей надобности отправки
    filtered_data = data[(data['deficit'] > 0) & (data['available'] > 0)].copy()
    filtered_data.sort_values(['priority', 'deficit'], ascending=[False, False], inplace=True)

    distribution = []
    rc_stock = filtered_data.groupby('product_id')['available'].first().to_dict()
    
    for _, row in filtered_data.iterrows():
        product_id = row['product_id']
        deficit = row['deficit']
        # проверка на остаток в рц и расчет количества отправки
        if rc_stock.get(product_id, 0) > 0:
            to_distribute = min(deficit, rc_stock[product_id])
            
            if to_distribute > 0:
                distribution.append({
                    'product_id': product_id,
                    'branch_id': row['branch_id'],
                    'quantity_to ship': to_distribute,
                    'deficit': deficit,
                    'available': row['available'],
                    'priority': row['priority'],
                    'logdays': row['logdays']
                })
                rc_stock[product_id] -= to_distribute

    result_df = pd.DataFrame(distribution)
    result_df.to_csv('result_df', index= False, header= True)
    return result_df

In [ ]:
distribution_algorithm()

C:\Users\Пользователь\AppData\Local\Temp\ipykernel_8296\3767737589.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


,product_id,branch_id,quantity_to ship,deficit,available,priority,logdays
0,01e7c50d-9bc9-11ea-a20e-00155df1b805,01f753c6-d3d3-11ed-905c-00155d32db05,68.0,1050.0,68.0,3.0,7.0
1,022cdc75-6fe0-11ef-9264-0050569d9cb8,01f753c6-d3d3-11ed-905c-00155d32db05,40.0,1049.0,40.0,3.0,7.0
2,01866436-91e2-11ef-9298-0050569d0a4d,01f753c6-d3d3-11ed-905c-00155d32db05,1.0,1043.0,1.0,3.0,7.0
3,0132a22a-8b9e-11eb-a238-00155dd2ff05,01f753c6-d3d3-11ed-905c-00155d32db05,34.0,1036.0,34.0,3.0,7.0
4,01f476d9-ad0d-11e9-b324-00155d03330f,01f753c6-d3d3-11ed-905c-00155d32db05,64.0,1036.0,64.0,3.0,7.0
...,...,...,...,...,...,...,...
205,04a026dd-6e2d-11ee-915f-00155d8ed20b,01f753c6-d3d3-11ed-905c-00155d32db05,6.0,21.0,6.0,3.0,7.0
206,04b0a591-bbd3-11ed-90ac-00155d8ed20c,01f753c6-d3d3-11ed-905c-00155d32db05,3.0,21.0,3.0,3.0,7.0
207,0176f7d7-0629-11f0-9300-0050569d9cb8,01f753c6-d3d3-11ed-905c-00155d32db05,7.0,13.0,7.0,3.0,7.0
208,02c6c803-ae69-11ed-909d-00155d8ed20c,01f753c6-d3d3-11ed-905c-00155d32db05,6.0,6.0,53.0,3.0,7.0
